In [11]:
import re
import codecs
import jieba
from zhihu import Question

为什么大多数高中宁愿学习衡中，而不愿学习理念更加先进的人大附中等？https://www.zhihu.com/question/47145647

In [12]:
q = Question('59436426')

In [13]:
stop_words = []
with codecs.open('stop_words.txt','r',encoding='utf-8') as f:
    stop_words = f.readlines()
stop_words = [sw.strip() for sw in stop_words]

In [14]:
def tokenize_with_rsw(string):
    return ' '.join([word for word in jieba.cut(string) if not word in stop_words and len(word) > 1])

In [15]:
if not q.is_json_exist():
    q.get_data_from_api()
    q.save_to_json()
q.get_data_from_json()
title_answers = []
title_answers.append(tokenize_with_rsw(re.sub('<[^>]+>','',q.questions['title'])))
for a in q.answers['items']:
    title_answers.append(tokenize_with_rsw(a['content']))

Building prefix dict from the default dictionary ...
Loading model from cache c:\users\lxp\appdata\local\temp\jieba.cache
Loading model cost 0.404 seconds.
Prefix dict has been built succesfully.


In [16]:
from __future__ import print_function

In [17]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [18]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### TF-IDF

In [19]:
docs_raw = title_answers
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words,
                                token_pattern = u'[\u4E00-\u9FFF]+',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(105, 49)


In [20]:
lda_tfidf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [21]:
pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     32.546143        1       1 -0.027863  0.000695
7      23.516010        1       2  0.098065 -0.019821
1       7.104627        1       3 -0.043111 -0.064605
10      4.481828        1       4 -0.009046  0.040741
0       4.076418        1       5 -0.001554  0.019811
3       3.761749        1       6 -0.017043  0.024049
19      2.233032        1       7  0.000545  0.012127
17      2.201386        1       8 -0.016158  0.007297
9       2.074992        1       9  0.003380  0.008182
4       1.967955        1      10 -0.003060 -0.003719
16      1.844416        1      11 -0.007860 -0.012270
6       1.841924        1      12  0.005315 -0.006930
5       1.690341        1      13  0.008302 -0.002348
14      1.690341        1      14 -0.002728  0.003424
8       1.690341        1      15 -0.000483 -0.000999
15      1.455700        1      16  0.008363  0.005214
13      1.455700        1      17 -0.000049 -0.001937
2       1.455700        1      18  0.000872 -0.002868
11      1.455700        1      19  0.002335 -0.004171
12      1.455700        1      20  0.001778 -0.001872, topic_info=     Category      Freq  Term     Total  loglift  logprob
term                                                     
40    Default  5.000000   社会学  5.000000  30.0000  30.0000
18    Default  6.000000    学生  6.000000  29.0000  29.0000
35    Default  6.000000    校长  6.000000  28.0000  28.0000
22    Default  6.000000    就业  6.000000  27.0000  27.0000
15    Default  7.000000    大学  7.000000  26.0000  26.0000
17    Default  5.000000    学校  5.000000  25.0000  25.0000
23    Default  4.000000    工作  4.000000  24.0000  24.0000
42    Default  6.000000    罗俊  6.000000  23.0000  23.0000
36    Default  7.000000    毕业  7.000000  22.0000  22.0000
7     Default  3.000000    人文  3.000000  21.0000  21.0000
1     Default  3.000000    一点  3.000000  20.0000  20.0000
39    Default  6.000000    社会  6.000000  19.0000  19.0000
29    Default  4.000000    教育  4.000000  18.0000  18.0000
27    Default  3.000000    改革  3.000000  17.0000  17.0000
32    Default  6.000000    本科  6.000000  16.0000  16.0000
4     Default  6.000000    中大  6.000000  15.0000  15.0000
12    Default  4.000000    发展  4.000000  14.0000  14.0000
41    Default  5.000000  社会工作  5.000000  13.0000  13.0000
28    Default  3.000000    政府  3.000000  12.0000  12.0000
38    Default  4.000000    真的  4.000000  11.0000  11.0000
31    Default  3.000000    未来  3.000000  10.0000  10.0000
48    Default  4.000000    高校  4.000000   9.0000   9.0000
2     Default  3.000000  专业对口  3.000000   8.0000   8.0000
0     Default  3.000000    一句  3.000000   7.0000   7.0000
37    Default  3.000000    相关  3.000000   6.0000   6.0000
3     Default  3.000000    中国  3.000000   5.0000   5.0000
5     Default  3.000000  中山大学  3.000000   4.0000   4.0000
9     Default  3.000000    全国  3.000000   3.0000   3.0000
24    Default  3.000000    希望  3.000000   2.0000   2.0000
19    Default  3.000000    学科  3.000000   1.0000   1.0000
...       ...       ...   ...       ...      ...      ...
10    Topic20  0.064671    几个  3.479444   0.2444  -3.8705
31    Topic20  0.061566    未来  3.387391   0.2220  -3.9197
28    Topic20  0.057577    政府  3.192547   0.2142  -3.9867
30    Topic20  0.060247    方向  3.353117   0.2105  -3.9413
0     Topic20  0.059475    一句  3.312909   0.2097  -3.9542
7     Topic20  0.062630    人文  3.489597   0.2094  -3.9026
21    Topic20  0.059076    对口  3.310260   0.2037  -3.9610
5     Topic20  0.070089  中山大学  3.973571   0.1920  -3.7900
27    Topic20  0.062821    改革  3.626026   0.1741  -3.8995
16    Topic20  0.058777    学习  3.535680   0.1328  -3.9660
19    Topic20  0.061520    学科  3.701043   0.1327  -3.9204
9     Topic20  0.065192    全国  3.964401   0.1219  -3.8625
6     Topic20  0.064274    事情  4.006841   0.0971  -3.8766
33    Topic20  0.065830  本科专业  4.126011   0.0917  -3.8527
24    Topic20  0.055024    希望  3.449988   0

### TF

In [22]:
tf_vectorizer = CountVectorizer(stop_words=stop_words,
                                token_pattern = u'[\u4E00-\u9FFF]+',
                                max_df = 0.5, 
                                min_df = 10)
dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(105, 49)


In [23]:
lda_tf = LatentDirichletAllocation(n_topics=20, random_state=0)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_jobs=1, n_topics=20, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [24]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     24.406666        1       1 -0.054531 -0.081505
5      13.254317        1       2  0.064457 -0.038942
19     10.781519        1       3  0.077102 -0.023931
2       8.580913        1       4  0.021753  0.079988
16      7.813146        1       5  0.037002  0.092087
13      7.724345        1       6  0.092034 -0.066293
7       6.055904        1       7  0.109910 -0.047187
4       4.572153        1       8 -0.180242 -0.112283
1       4.447541        1       9 -0.163561  0.124753
9       3.636387        1      10  0.044783  0.055282
17      2.996965        1      11 -0.000437 -0.025120
0       2.897592        1      12  0.041753  0.076082
15      0.980951        1      13 -0.015805 -0.045735
10      0.293849        1      14 -0.008193 -0.004297
8       0.259626        1      15 -0.006838  0.001118
11      0.259626        1      16 -0.006922  0.009415
3       0.259626        1      17 -0.009257  0.000728
12      0.259626        1      18 -0.011696 -0.000720
14      0.259626        1      19 -0.012593 -0.001644
6       0.259626        1      20 -0.018718  0.008202, topic_info=     Category       Freq  Term      Total  loglift  logprob
term                                                       
40    Default  49.000000   社会学  49.000000  30.0000  30.0000
39    Default  67.000000    社会  67.000000  29.0000  29.0000
15    Default  59.000000    大学  59.000000  28.0000  28.0000
36    Default  33.000000    毕业  33.000000  27.0000  27.0000
29    Default  40.000000    教育  40.000000  26.0000  26.0000
41    Default  50.000000  社会工作  50.000000  25.0000  25.0000
18    Default  79.000000    学生  79.000000  24.0000  24.0000
12    Default  43.000000    发展  43.000000  23.0000  23.0000
22    Default  49.000000    就业  49.000000  22.0000  22.0000
23    Default  39.000000    工作  39.000000  21.0000  21.0000
17    Default  51.000000    学校  51.000000  20.0000  20.0000
32    Default  64.000000    本科  64.000000  19.0000  19.0000
38    Default  38.000000    真的  38.000000  18.0000  18.0000
35    Default  35.000000    校长  35.000000  17.0000  17.0000
14    Default  33.000000    培养  33.000000  16.0000  16.0000
13    Default  41.000000    同学  41.000000  15.0000  15.0000
5     Default  27.000000  中山大学  27.000000  14.0000  14.0000
4     Default  75.000000    中大  75.000000  13.0000  13.0000
43    Default  27.000000    老师  27.000000  12.0000  12.0000
20    Default  26.000000    学院  26.000000  11.0000  11.0000
8     Default  20.000000    价值  20.000000  10.0000  10.0000
48    Default  22.000000    高校  22.000000   9.0000   9.0000
19    Default  21.000000    学科  21.000000   8.0000   8.0000
21    Default  24.000000    对口  24.000000   7.0000   7.0000
6     Default  24.000000    事情  24.000000   6.0000   6.0000
27    Default  24.000000    改革  24.000000   5.0000   5.0000
28    Default  18.000000    政府  18.000000   4.0000   4.0000
37    Default  16.000000    相关  16.000000   3.0000   3.0000
11    Default  19.000000    原因  19.000000   2.0000   2.0000
16    Default  18.000000    学习  18.000000   1.0000   1.0000
...       ...        ...   ...        ...      ...      ...
16    Topic20   0.082556    学习  18.657328   0.5332  -3.8275
30    Topic20   0.071406    方向  16.664707   0.5010  -3.9726
3     Topic20   0.070908    中国  16.967021   0.4760  -3.9796
9     Topic20   0.075496    全国  18.446843   0.4551  -3.9169
19    Topic20   0.085369    学科  21.214279   0.4382  -3.7940
44    Topic20   0.067042    裁撤  17.228971   0.4047  -4.0356
46    Topic20   0.067871    选择  17.525634   0.3999  -4.0233
42    Topic20   0.073898    罗俊  19.221843   0.3926  -3.9383
33    Topic20   0.072834  本科专业  19.588901   0.3592  -3.9528
8     Topic20   0.074737    价值  20.138336   0.3573  -3.9270
24    Topic20   0.072466    希望  19.546683   0.3562  -3.9578
6     Topic20   0.087411    事情  24.273896   0.3272  -3.7703
11    Topic20   0.069725    原因  19.375920   0.3265  -3.9964
34    Top

### MDS

In [25]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='mmds')

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
18     24.406666        1       1  0.112437  0.047554
5      13.254317        1       2  0.091984 -0.067172
19     10.781519        1       3 -0.125342  0.022896
2       8.580913        1       4 -0.143374 -0.053370
16      7.813146        1       5 -0.076723 -0.048139
13      7.724345        1       6 -0.007475 -0.146271
7       6.055904        1       7 -0.070681 -0.128678
4       4.572153        1       8  0.190686  0.126327
1       4.447541        1       9  0.000151  0.207677
9       3.636387        1      10  0.053907 -0.132104
17      2.996965        1      11  0.060850 -0.046556
0       2.897592        1      12 -0.089580  0.083769
15      0.980951        1      13  0.041157  0.040202
10      0.293849        1      14 -0.007915  0.022276
8       0.259626        1      15 -0.004179  0.006342
11      0.259626        1      16 -0.008314  0.009525
3       0.259626        1      17 -0.004151  0.012041
12      0.259626        1      18 -0.004003  0.015040
14      0.259626        1      19 -0.005220  0.014515
6       0.259626        1      20 -0.004215  0.014125, topic_info=     Category       Freq  Term      Total  loglift  logprob
term                                                       
40    Default  49.000000   社会学  49.000000  30.0000  30.0000
39    Default  67.000000    社会  67.000000  29.0000  29.0000
15    Default  59.000000    大学  59.000000  28.0000  28.0000
36    Default  33.000000    毕业  33.000000  27.0000  27.0000
29    Default  40.000000    教育  40.000000  26.0000  26.0000
41    Default  50.000000  社会工作  50.000000  25.0000  25.0000
18    Default  79.000000    学生  79.000000  24.0000  24.0000
12    Default  43.000000    发展  43.000000  23.0000  23.0000
22    Default  49.000000    就业  49.000000  22.0000  22.0000
23    Default  39.000000    工作  39.000000  21.0000  21.0000
17    Default  51.000000    学校  51.000000  20.0000  20.0000
32    Default  64.000000    本科  64.000000  19.0000  19.0000
38    Default  38.000000    真的  38.000000  18.0000  18.0000
35    Default  35.000000    校长  35.000000  17.0000  17.0000
14    Default  33.000000    培养  33.000000  16.0000  16.0000
13    Default  41.000000    同学  41.000000  15.0000  15.0000
5     Default  27.000000  中山大学  27.000000  14.0000  14.0000
4     Default  75.000000    中大  75.000000  13.0000  13.0000
43    Default  27.000000    老师  27.000000  12.0000  12.0000
20    Default  26.000000    学院  26.000000  11.0000  11.0000
8     Default  20.000000    价值  20.000000  10.0000  10.0000
48    Default  22.000000    高校  22.000000   9.0000   9.0000
19    Default  21.000000    学科  21.000000   8.0000   8.0000
21    Default  24.000000    对口  24.000000   7.0000   7.0000
6     Default  24.000000    事情  24.000000   6.0000   6.0000
27    Default  24.000000    改革  24.000000   5.0000   5.0000
28    Default  18.000000    政府  18.000000   4.0000   4.0000
37    Default  16.000000    相关  16.000000   3.0000   3.0000
11    Default  19.000000    原因  19.000000   2.0000   2.0000
16    Default  18.000000    学习  18.000000   1.0000   1.0000
...       ...        ...   ...        ...      ...      ...
16    Topic20   0.082556    学习  18.657328   0.5332  -3.8275
30    Topic20   0.071406    方向  16.664707   0.5010  -3.9726
3     Topic20   0.070908    中国  16.967021   0.4760  -3.9796
9     Topic20   0.075496    全国  18.446843   0.4551  -3.9169
19    Topic20   0.085369    学科  21.214279   0.4382  -3.7940
44    Topic20   0.067042    裁撤  17.228971   0.4047  -4.0356
46    Topic20   0.067871    选择  17.525634   0.3999  -4.0233
42    Topic20   0.073898    罗俊  19.221843   0.3926  -3.9383
33    Topic20   0.072834  本科专业  19.588901   0.3592  -3.9528
8     Topic20   0.074737    价值  20.138336   0.3573  -3.9270
24    Topic20   0.072466    希望  19.546683   0.3562  -3.9578
6     Topic20   0.087411    事情  24.273896   0.3272  -3.7703
11    Topic20   0.069725    原因  19.375920   0.3265  -3.9964
34    Top

In [26]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer, mds='tsne')

PreparedData(topic_coordinates=            Freq  cluster  topics           x           y
topic                                                    
18     24.406666        1       1  -70.801975  -84.217016
5      13.254317        1       2   -3.394994 -158.787553
19     10.781519        1       3  -64.827103   62.364458
2       8.580913        1       4    3.071911    8.195466
16      7.813146        1       5  102.876332   17.743188
13      7.724345        1       6   72.814379  -48.221674
7       6.055904        1       7    4.776774  -72.773404
4       4.572153        1       8  115.674049  140.341484
1       4.447541        1       9  -76.142275   -8.587337
9       3.636387        1      10 -153.874088  -53.283125
17      2.996965        1      11  -11.949076  123.080420
0       2.897592        1      12 -155.070573   38.441442
15      0.980951        1      13 -283.647321  -27.333641
10      0.293849        1      14  347.107465   27.258881
8       0.259626        1      15 -113.143205  130.516112
11      0.259626        1      16   47.678360   70.033078
3       0.259626        1      17 -103.394501 -154.766228
12      0.259626        1      18  153.169347  -74.179443
14      0.259626        1      19  182.117738   27.021378
6       0.259626        1      20   80.581780 -135.400613, topic_info=     Category       Freq  Term      Total  loglift  logprob
term                                                       
40    Default  49.000000   社会学  49.000000  30.0000  30.0000
39    Default  67.000000    社会  67.000000  29.0000  29.0000
15    Default  59.000000    大学  59.000000  28.0000  28.0000
36    Default  33.000000    毕业  33.000000  27.0000  27.0000
29    Default  40.000000    教育  40.000000  26.0000  26.0000
41    Default  50.000000  社会工作  50.000000  25.0000  25.0000
18    Default  79.000000    学生  79.000000  24.0000  24.0000
12    Default  43.000000    发展  43.000000  23.0000  23.0000
22    Default  49.000000    就业  49.000000  22.0000  22.0000
23    Default  39.000000    工作  39.000000  21.0000  21.0000
17    Default  51.000000    学校  51.000000  20.0000  20.0000
32    Default  64.000000    本科  64.000000  19.0000  19.0000
38    Default  38.000000    真的  38.000000  18.0000  18.0000
35    Default  35.000000    校长  35.000000  17.0000  17.0000
14    Default  33.000000    培养  33.000000  16.0000  16.0000
13    Default  41.000000    同学  41.000000  15.0000  15.0000
5     Default  27.000000  中山大学  27.000000  14.0000  14.0000
4     Default  75.000000    中大  75.000000  13.0000  13.0000
43    Default  27.000000    老师  27.000000  12.0000  12.0000
20    Default  26.000000    学院  26.000000  11.0000  11.0000
8     Default  20.000000    价值  20.000000  10.0000  10.0000
48    Default  22.000000    高校  22.000000   9.0000   9.0000
19    Default  21.000000    学科  21.000000   8.0000   8.0000
21    Default  24.000000    对口  24.000000   7.0000   7.0000
6     Default  24.000000    事情  24.000000   6.0000   6.0000
27    Default  24.000000    改革  24.000000   5.0000   5.0000
28    Default  18.000000    政府  18.000000   4.0000   4.0000
37    Default  16.000000    相关  16.000000   3.0000   3.0000
11    Default  19.000000    原因  19.000000   2.0000   2.0000
16    Default  18.000000    学习  18.000000   1.0000   1.0000
...       ...        ...   ...        ...      ...      ...
16    Topic20   0.082556    学习  18.657328   0.5332  -3.8275
30    Topic20   0.071406    方向  16.664707   0.5010  -3.9726
3     Topic20   0.070908    中国  16.967021   0.4760  -3.9796
9     Topic20   0.075496    全国  18.446843   0.4551  -3.9169
19    Topic20   0.085369    学科  21.214279   0.4382  -3.7940
44    Topic20   0.067042    裁撤  17.228971   0.4047  -4.0356
46    Topic20   0.067871    选择  17.525634   0.3999  -4.0233
42    Topic20   0.073898    罗俊  19.221843   0.3926  -3.9383
33    Topic20   0.072834  本科专业  19.588901   0.3592  -3.9528
8     Topic20   0.074737    价值  20.138336   0.3573  -3.9270
24    Topic20   0.072466    希望  19.546683   0.3562  -3.9578
6     Topic20   0.087411    事情  24.273896